In [54]:
import h5py
import glob
import scipy.io
from scipy.stats import stats
from sklearn.metrics import confusion_matrix
from sklearn.svm import NuSVC
import numpy as np
import random
from settings import *
# path = '/data/HBN/test/fmriprep_output/fmriprep/PythonData/'

In [55]:
from brainiak import isfc

In [93]:
subord = glob.glob(h5path+'sub*.h5')
with h5py.File(h5path+'ISC.h5') as hf:
    string_dt = h5py.special_dtype(vlen=str)
    hf.create_dataset("subord",data=np.array(subord).astype('|S70'),dtype=string_dt)
for task in ['DM','TP']:
    data = []
    for sub in subord:
        f = h5py.File(sub, 'r')
        D = {}
        for hem in ['L','R']:
            D[hem] = f[task][hem][:]
        data.append(np.concatenate(
            [D['L'], D['R']], axis=0))
    data = np.dstack(data)
    ISC = isfc.isc(data,collapse_subj=True)
    ISC_persubj = isfc.isc(data,collapse_subj=False)
    with h5py.File(h5path+'ISC.h5') as hf:
        grp = hf.create_group(task)
        grp.create_dataset('ISC', data=ISC)
        grp.create_dataset('ISC_persubj', data=ISC_persubj)

In [93]:
%matplotlib widget
import matplotlib.pyplot as plt
# load ISC (CHANGE PATH LATER)
iscf = h5py.File(
    '/data/HBN/test/fmriprep_output/fmriprep/PythonData/ISC.h5', 'r')
# pick which random coords to display:
f = h5py.File('/data/Schema/intact/fsaverage6_adj.h5', 'r')
dispcoordsLH = f['lhinflatedcoords'][:]
randcoords = random.sample(range(len(dispcoordsLH[1])), 5000)
dispcoordsLH = dispcoordsLH[:,randcoords]
dispcoordsRH = f['rhinflatedcoords'][:]
dispcoordsRH = dispcoordsRH[:,randcoords]

#fig1 = plt.figure()
fig2 = plt.figure()

# f = h5py.File(h5path+'ISC.h5', 'r')
f = h5py.File(
    '/data/HBN/test/fmriprep_output/fmriprep/PythonData/ISC.h5', 'r')
Ages = []
Sexes = []
for s in f['subord'][:]:
    fs = h5py.File(s,'r')
    Ages.append(fs['Pheno']['Age'].value)
    Sexes.append(fs['Pheno']['Sex'].value)


for idx,task in enumerate(['DM','TP']):
    ISC = np.nan_to_num(f[task]['ISC'][:])
    ISC_persubj = np.nanmean(f[task]['ISC_persubj'][:],axis=0)
    for a in 
    # For SUMA visualization:
    np.savetxt(h5path+task+'ISC.txt',ISC)
    np.savetxt(h5path+task+'ISC_LH.txt',ISC[:len(ISC)//2])
    np.savetxt(h5path+task+'ISC_RH.txt',ISC[len(ISC)//2:])
    dispISCLH = ISC[:len(ISC)//2][randcoords] # For LH
    dispISCRH = ISC[len(ISC)//2:][randcoords] # For LH
    # plot ISC values across cortex
    axrh = fig1.add_subplot(2,2,idx+1, projection='3d')
    x,y,z = dispcoordsRH
    brainrh = axrh.scatter3D(x,y,z,c=dispISCRH, cmap='bwr')
    fig1.colorbar(brainrh)
    axrh.set_title(task + ' RH')
    
    axlh = fig1.add_subplot(2,2,idx+3, projection='3d')
    x,y,z = dispcoordsLH
    brainlh = axlh.scatter3D(x,y,z,c=dispISCLH, cmap='bwr');
    fig1.colorbar(brainlh)
    axlh.set_title(task + ' LH')
    # plot per-subject ISC values by age
    ax = fig2.add_subplot(2,2,idx+1)
    ax.hist(Ages,weights=ISC_persubj)
    ax.set_title(task + ' ISC by age')
    # plot per-subject ISC values by sex
    ax = fig2.add_subplot(2,2,idx+3)
    ax.hist(Sexes,weights=ISC_persubj)
    ax.set_title(task + ' ISC by sex')
    # plot matrix of per-subject ISCs

FigureCanvasNbAgg()

In [6]:
%matplotlib widget
fig = plt.figure()
#ax = plt.axes(projection='3d')
ax = fig.gca(projection='3d')

x,y,z = dispcoordsRH
brain = ax.scatter3D(x,y,z,c=dispISCRH, cmap='bwr');
plt.colorbar(brain)

FigureCanvasNbAgg()